In [2]:
import importlib
importlib.reload

<function importlib.reload>

In [1]:
# Mengakses Google Drive User
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# import library NLTK 
import nltk
nltk.download('stopwords')

In [3]:
# import library lainnya
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences